# Image Generation with OpenVINO GenAI

## Install dependencies

In [ ]:
!pip install openvino-genai optimum-intel nncf huggingface diffusers --extra-index-url https://download.pytorch.org/whl/cpu

## Download and convert the model

In [ ]:
from optimum.exporters.openvino.convert import export_tokenizer
from optimum.intel import OVPipelineForText2Image

output_dir = "dreamshaper-v7-lcm"

pipeline = OVPipelineForText2Image.from_pretrained("SimianLuo/LCM_Dreamshaper_v7", export=True)
pipeline.save_pretrained(output_dir)
export_tokenizer(pipeline.tokenizer, output_dir + "/tokenizer")

## Query devices

In [ ]:
import openvino as ov

core = ov.Core()
available_devices = core.available_devices

print(available_devices)
print([core.get_property(device, "FULL_DEVICE_NAME") for device in available_devices])

## Run inference

In [ ]:
import ipywidgets as widgets

# Select the model type
device_dropdown = widgets.Dropdown(
    options=available_devices,
    value="CPU",
    description="Device:"
)
device_dropdown

In [ ]:
import openvino_genai as ov_genai

pipe = ov_genai.Text2ImagePipeline(output_dir, device_dropdown.value)

In [ ]:
import numpy as np
from PIL import Image

prompt = "Cartoon-style adventurer child with a backpack and a treasure map"

seed = np.random.randint(2**32)
image_tensor = pipe.generate(prompt, rng_seed=seed, width=512, height=512, num_inference_steps=10, num_images_per_prompt=1)

image = Image.fromarray(image_tensor.data[0])
image